# Connecting to database

In [4]:

%pip install mysql-connector-python
import pandas as pd
import glob
import mysql.connector

# List of CSV file paths
csv_files = ["red_bus_details_data.csv"]

# Read each CSV file into a DataFrame and concatenate them into a single DataFrame
df_list = [pd.read_csv(file) for file in csv_files]
df = pd.concat(df_list, ignore_index=True)


# Extract the digits (0-9)
df['Seat_Availability'] = df['Seat_Availability'].str.extract('(\d+)')

# Drop null values
df = df.dropna()

# Python MySQL connection
myconnection = mysql.connector.connect(
    host='localhost',
    user='root',
    passwd='gayu7102',
    database="redbus_details"
)

# Replace pandas dtypes with MySQL compatible types
dtype_mapping = {
    "float64": "float",
    "object": "text",
    "int64": "int"
}

# Create the table with the appropriate column types
columns_and_types = ", ".join(f"{col} {dtype_mapping[str(dtype)]}"
                              for col, dtype in zip(df.columns, df.dtypes))
table_name = "bus_routes_data"
create_table_query = f"CREATE TABLE IF NOT EXISTS {table_name} ({columns_and_types})"
cursor = myconnection.cursor()
cursor.execute(create_table_query)

# Insert data into the table
insert_query = f"INSERT INTO {table_name} VALUES ({', '.join(['%s'] * len(df.columns))})"
for i in range(len(df)):
    cursor.execute(insert_query, tuple(df.iloc[i]))

# Commit the transaction
myconnection.commit()

# Close the connection
cursor.close()
myconnection.close()

print("Data inserted successfully!")

Note: you may need to restart the kernel to use updated packages.
Data inserted successfully!
